In [2]:
import os
os.chdir("../")
%pwd

'f:\\Machine_Learning\\full_stack_data_science-ineuron\\Computer Vision\\CNNProjectStructure'

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
# config-entity.py
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

#### Configuration of input of Prepare_base_model component

In [5]:
# configuration.py
from CNNClassifier.constants import *
from CNNClassifier.utils.utils import read_yaml, create_directory


class ConfigurationManager:
    # Class to map config_entity.py with config.yaml file

    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        # Load parameters for stage02_prepare_base_model.py component
        config = self.config.prepare_base_model

        # artifacts/prepare_base_model folder created
        create_directory([config.root_dir])

        # Assign values from config.yaml file to PrepareBaseModelConfig dataclass in config_entity.
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

#### Definition of pipeline functionalities

In [6]:
import tensorflow as tf

In [8]:
# stage_02_prepare_base_model.py
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        # Load the VGG16 base model
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        # Save the loaded base model
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        '''
            This function is used to set various parameters of the base model and finally prepare the full model to be used for training.
        '''
        if freeze_all:
            model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,                       # get the base model
            classes=self.config.params_classes,     # number of classes at output
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        # Save the updated base model/final model
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

#### Executing the Pipeline

In [10]:
try:
    config = ConfigurationManager()

# Load prepare_base_model parameter values from config.yaml to PrepareBaseModelConfig dataclass.
    prepare_base_model_config = config.get_prepare_base_model_config()

# logger.info("Stage 02: Prepare Base Model started.")

# Loading input parameters from PrepareBaseModelConfig dataclass to stage_02_prepare_base_model.py file.
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
# logger.info("Stage 02: Base VGG16 model created successfully. Base model is saved.")
    prepare_base_model.update_base_model()
# logger.info("Stage 02: Base VGG16 model updated successfully. Final model is saved.")
# logger.info("Stage 02: Prepare Base Model completed successfully.")
except Exception as e:
    raise e

[2024-01-11 21:13:18,062: INFO: utils]: Directory created at artifacts/prepare_base_model
58889256/58889256 [==============================] - 24s 0us/step
[2024-01-11 21:13:44,324: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


f:\Machine_Learning\full_stack_data_science-ineuron\Computer Vision\CNNProjectStructure\env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     